In [ ]:
%load_ext autoreload
%autoreload 1

%aimport basics
%aimport nn_config
%aimport enc_dec


from basics import *
from nn_config import *
from enc_dec import *
%matplotlib inline

In [ ]:
xp = cuda.cupy if gpuid >= 0 else np

In [ ]:
text_data = pickle.load(open(text_data_dict, "rb"))

In [ ]:
model = SpeechEncoderDecoder(SPEECH_DIM, vocab_size_en, num_layers_enc, num_layers_dec,
                               hidden_units, gpuid, attn=use_attn)
if gpuid >= 0:
    cuda.get_device(gpuid).use()
    model.to_gpu()

In [ ]:
log_train_fil_name, text_fname, dev_fname, test_fname

In [ ]:
%aimport nmt_trials
from nmt_trials import *

In [ ]:
buckets, bucket_lengths = populate_buckets()

In [ ]:
25*32, 24*35

In [ ]:
BATCH_SIZE, BATCH_SIZE

In [ ]:
_, en, sf = get_data_item('053.181')
xp.isnan(xp.sum(sf))

In [ ]:
# for sp_fil in text_data["train"].keys():
#     _, en, sf = get_data_item(sp_fil)
#     if xp.isnan(xp.sum(sf)) == True:
#         print(sp_fil)

In [ ]:
bs = [34]
subset_buckets = []
subset_bucket_lengths = {}
for i, b in enumerate(bs):
    subset_buckets.append(buckets[b])
    subset_bucket_lengths[i]=bucket_lengths[b]

In [ ]:
subset_bucket_lengths, len(subset_buckets)

In [ ]:
subset_num_training = 1000
subset_batch_size = 50

In [ ]:
batch_training(subset_num_training, subset_batch_size, subset_buckets, subset_bucket_lengths, SPEECH_BUCKET_WIDTH, 0)

In [ ]:
model.L0_enc.lateral.W.data[:2,:5]

In [ ]:
model.enc_states.shape, model.L1_dec.h.shape

In [ ]:
weights = F.batch_matmul(model.enc_states, model[model.lstm_dec[-1]].h)

In [ ]:
weights = F.batch_matmul(F.swapaxes(model.enc_states, 0, 1), model[model.lstm_dec[-1]].h)

In [ ]:
F.reshape(weights,(weights.shape[0],weights.shape[1])).shape

In [ ]:
F.softmax(xp.array([[-0.01054492], [-0.00358161],[ 0.00333601]])).data
xp.array([[-0.01054492], [-0.00358161],[ 0.00333601]]).shape

In [ ]:
weights.data, weights.shape

In [ ]:
model.mask.shape, model.minf.shape

In [ ]:
F.softmax(F.reshape(weights,(weights.shape[0],weights.shape[1]))).data

In [ ]:
alphas = F.softmax(weights,True)
alphas.data, alphas.shape

In [ ]:
ha = F.swapaxes(model.enc_states, 0,1)
haha = F.swapaxes(ha, 1,2)

In [ ]:
model.enc_states.shape, haha.shape

In [ ]:
F.batch_matmul(haha, alphas).shape, F.squeeze(F.batch_matmul(haha, alphas),axis=2).shape

In [ ]:
cv = F.reshape(F.batch_matmul(F.swapaxes(model.enc_states, 2, 1), alphas), shape=(batch_size, n_units))

In [ ]:
# predict(s=0, num=1, cat="train", display=True, plot=False, p_filt=0, r_filt=0)

In [ ]:
# print(b" ".join(get_ids(text_data["train"]["041.004"]["en"])[0]))

In [ ]:
# forward_states = model[model.lstm_enc[-1]].h
# backward_states = model[model.lstm_rev_enc[-1]].h

In [ ]:
# model.enc_states = F.concat((forward_states, backward_states), axis=1)

In [ ]:
# train_loop(num_training=1000, num_epochs=1)

In [ ]:
wavs = []
wavs.append(xp.load(os.path.join(speech_dir, "041.001"+speech_extn)))
wavs.append(xp.load(os.path.join(speech_dir, "041.004"+speech_extn)))

In [ ]:
[l.shape for l in wavs]

In [ ]:
zero_arr = xp.zeros((3,120), dtype=xp.float32)

In [ ]:
wavs[1].dtype, zero_arr.dtype

In [ ]:
# xp.vstack((wavs[1], zero_arr))
wavs[1].shape, zero_arr.shape

In [ ]:
wavs.append(pad_array(wavs[1], 1000, at_start=True))

In [ ]:
xp.concatenate((wavs[1], zero_arr), axis=0).shape

In [ ]:
wavs[1][:3].shape

In [ ]:
xp.pad(wavs[1], pad_width=(0,100), mode='constant', constant_values=0)

In [ ]:
def display_buckets(bucket_lengths, width_b = SPEECH_BUCKET_WIDTH):
    headings = ("ix", "len b", "num", "max fr", "avg fr", "max en", "avg en")
    print("{0:3s} | {1:5s} | {2:5s} | {3:6s} | {4:8s} | {5:6s} | {6:8s}".format(*headings))
    print("\n".join(["{0:3d} | {1:5d} | {2:5d} | {3:6d} | {4:8.0f} | {5:6d} | {6:8.0f}".format(i[0], (i[0]+1)*width_b, *i[1]) for i in list(bucket_lengths.items())]))


def populate_buckets(width_b = SPEECH_BUCKET_WIDTH, 
                     num_b = SPEECH_NUM_BUCKETS, 
                     speech=True, 
                     num_sent=NUM_TRAINING_SENTENCES,
                     filname_b=speech_bucket_data_fname,
                     cat="train", display=False):
    
    buckets = [[] for i in range(num_b)]
    
    print("Splitting data into {0:d} buckets, each of width={1:d}".format(num_b, width_b))

    with tqdm(total=num_sent) as pbar:
        for i, sp_fil in enumerate(sorted(list(text_data[cat].keys()))[:num_sent]):

            fr_ids, en_ids, speech_feat = get_data_item(sp_fil, cat=cat)

            len_en = len(en_ids)
            len_fr = len(fr_ids)
            len_speech = len(speech_feat)
            
            indx_b = min(num_b-1, len_speech // width_b)

            buckets[indx_b].append((sp_fil, len_speech, len_fr, len_en))

            pbar.update(1)
    
    bucket_lengths = {i:(len(l), 
                    max(l, key=lambda t:t[2])[2],
                    np.mean([i[2]for i in l]),
                    max(l, key=lambda t:t[3])[3],
                    np.mean([i[3]for i in l]))
                     for i, l in enumerate(buckets)}

    if display:
        display_buckets(bucket_lengths)

    # Saving bucket data
    print("Saving bucket data")
    pickle.dump(buckets, open(filname_b, "wb"))
    return buckets, bucket_lengths

In [ ]:
b, bucket_lengths = populate_buckets(display=True)

In [ ]:
b[1][:5]

In [ ]:
haha = [0 for i in range(10)]

In [ ]:
d = haha[:30]
print(len(d))

In [ ]:
bucket_lengths